In [1]:
import tensorflow as tf
import numpy as np
import pickle

In [ ]:
# lines = open('/Users/Alan/Nuance/zh-parse-tool/corpus/TrainSet-eCarX-171019.txt', encoding='gbk').readlines()
# labels = [i.split()[0] for i in lines]
# labels = list(set(labels))
# label_map = np.load('./multi_label_map')
# def combine(data, key, index, labels):
#     string = []
#     for i in range(5):
#         temp = data[i][key][index]
#         string.append(label_map[i][np.argmax(temp)])
#     string = [i for i in string if i != 'NULL']
#     rebuild = '.'.join(string)
#     result = np.zeros((len(labels)))
#     result[labels.index(rebuild)] = 1.0
#     return result

# combined_targets = dict()
# combined_train_targets = dict()
# combined_valid_targets = dict()

# keyname = 'train_targets'
# keys = data_all[keyname][0].keys()
# for key in keys:
#     size = len(data_all[keyname][0][key])
#     combined_train_targets[key] = np.zeros((size, len(labels)))
#     for i in range(size):
#         combined_train_targets[key][i] = combine(data_all[keyname], key, 0, labels)
        
# keyname = 'valid_targets'
# keys = data_all[keyname][0].keys()
# for key in keys:
#     size = len(data_all[keyname][0][key])
#     combined_valid_targets[key] = np.zeros((size, len(labels)))
#     for i in range(size):
#         combined_valid_targets[key][i] = combine(data_all[keyname], key, 0, labels)
        
# combined_targets['train_targets'] = combined_train_targets
# combined_targets['valid_targets'] = combined_valid_targets
# pickle.dump(combined_targets, open('./rebuild_label.npz', 'wb'))

In [2]:
data = pickle.load(open('./multi_train_data', 'rb'))
label_all = pickle.load(open('./multi_train_label', 'rb'))
test_data = pickle.load(open('./multi_test_data', 'rb'))
test_label_all = pickle.load(open('./multi_test_label', 'rb'))
sizes = pickle.load(open('./multi_label_sizes', 'rb'))
rebuild_label = pickle.load(open('./rebuild_label.npz', 'rb'))

data_all = {
    'train_inputs': data,
    'train_targets': label_all,
    'valid_inputs': test_data,
    'valid_targets': test_label_all,
    'label_size': sizes
}

In [3]:
graph = tf.Graph()

In [4]:
class parameter(object):
    
    def __init__(self, shape):
        self.shape = shape
    
    def normal(self):
        return tf.Variable(tf.random_normal(self.shape))

class layer(object):
    
    def __init__(self, shape):
        raise NotImplementedError
        
    def __str__(self):
        raise NotImplementedError

    
    
class lstm_layer(layer):
    
    def __init__(self, inputs, n_units):
        
        self.n_units = n_units
        self.input_shape = inputs.shape
        self.outputs, self.states = tf.nn.dynamic_rnn(
            cell = tf.contrib.rnn.BasicLSTMCell(self.n_units),
            inputs = inputs,
            dtype = tf.float32)
        
    def __str__(self):
        print('|---------------[LSTM layer]---------------|')
        print('shape: ', self.input_shape, ' => ', self.outputs.shape)
        return '|_______________[LSTM layer]_______________|'
        
class affine_layer(layer):
    
    def __init__(self, inputs, shape, activation):
        
        self.shape = shape
        self.input_shape = inputs.shape
        self.activation = activation
        self.weights = parameter(self.shape).normal()
        self.biases = parameter([self.shape[-1]]).normal()
        self.outputs = self.activation(tf.add(tf.matmul(inputs, self.weights), self.biases))

    def __str__(self):
        print('|---------------[Affine layer]---------------|')
        print('shape: ', self.input_shape, ' => ', self.outputs.shape)
        return '|_______________[Affine layer]_______________|'
    
class reshape_layer(layer):
    
    def __init__(self, inputs, shape):
        
        self.input_shape = inputs.shape
        self.outputs = tf.reshape(inputs, shape)

    def __str__(self):
        print('|---------------[Reshape layer]---------------|')
        print('shape: ', self.input_shape, ' => ', self.outputs.shape)
        return '|_______________[Reshape layer]_______________|'

In [19]:
class lstm_model(object):
    
    def __init__(self, data, model_id, n_affine = 1):
        
        self.frame_size = 16
        self.learning_rate = 1e-3
        self.data = data
        self.output_size = int(self.data['label_size'][model_id])
        self.hidden_unit1 = 256
#         self.graph = graph_name
#         print(self.output_size)
        
        
        with graph.as_default():
            
            with tf.variable_scope('single_model_'+str(model_id)):
                self.input_placeholder = tf.placeholder(tf.float32, [None, None, self.frame_size])
                self.target_placeholder = tf.placeholder(tf.float32, [None, self.output_size])
                self.batch_size_placeholder = tf.placeholder(tf.int32)

                self.layers = []
                self.layers.append(lstm_layer(self.input_placeholder, self.frame_size))
                self.layers.append(reshape_layer(self.layers[-1].outputs[:,-1,:], [-1, self.frame_size]))
    #             print(self.layers[-1].outputs)
                self.layers.append(affine_layer(self.layers[-1].outputs, [self.frame_size, self.hidden_unit1], tf.nn.relu))
                self.layers.append(affine_layer(self.layers[-1].outputs, [self.hidden_unit1, self.output_size], tf.identity))

                self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.layers[-1].outputs, labels=self.target_placeholder))
                self.optimizer = tf.train.AdamOptimizer
                self.training = self.optimizer(self.learning_rate).minimize(self.loss)
                self.accuracy = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(self.layers[-1].outputs, 1), tf.argmax(self.target_placeholder, 1)), tf.float32))
    #             self.number = self.target_placeholder.shape[0]
                
                self.sess = tf.Session()
                self.sess.run(tf.global_variables_initializer())
#             print(self.number)
            
    def train(self, iteration = 100, interval = 5):
           
        with graph.as_default():
            with tf.variable_scope('single_model_'+str(model_id)):
                for i in range(iteration):
                    accs = 0.0
                    errs = 0.0
                    ns = 0
                    for key, j in zip(self.data['train_inputs'], range(len(self.data['train_inputs']))):
                        train_inputs = np.array(self.data['train_inputs'][key])
                        train_targets = np.array(self.data['train_targets'][key])
                        batch_size = key
                        feed_dict = {self.input_placeholder: train_inputs, self.target_placeholder: train_targets, self.batch_size_placeholder: batch_size}
                        _, acc, err = self.sess.run([self.training, self.accuracy, self.loss], feed_dict = feed_dict)
                        accs += acc
                        errs += err
                        ns += train_inputs.shape[0]
                    accs /= ns
                    errs /= ns
                    print('Training Epoch: %d, acc: %f, loss: %f '%(i, accs, errs))
                if i % interval:
                    accs = 0.0
                    errs = 0.0
                    ns = 0
                    for key, j in zip(self.data['valid_inputs'], range(len(self.data['valid_inputs']))):
                        valid_inputs = np.array(self.data['valid_inputs'][key])
                        valid_targets = np.array(self.data['valid_targets'][key])
                        batch_size = key
                        feed_dict = {self.input_placeholder: valid_inputs, self.target_placeholder: valid_targets, self.batch_size_placeholder: batch_size}
                        _, acc, err = self.sess.run([self.training, self.accuracy, self.loss], feed_dict = feed_dict)
                        accs += acc
                        errs += err
                        ns += train_inputs.shape[0]
                    accs /= ns
                    errs /= ns
                    print('------------------------------------------------------')
                    print('Validation Epoch: %d, acc: %f, loss: %f '%(i, accs, errs))
                    print('------------------------------------------------------')
        
    def record(self):
        raise NotImplementedError
        
    def __str__(self):
        for layer in self.layers:
            print(layer)
        return ''
     

In [ ]:
# class multi_model(object):
    
#     def __init__(self, data, rebuild_target):
        
#         self.data = data
#         self.ids = np.arange(5)
#         self.data = dict()
#         self.graph = tf.Graph()
#         self.models = dict()
#         self.output_size = 30
#         self.hidden_unit1 = 128
#         self.learning_rate = 1e-3
#         self.rebuild_target = rebuild_target
#         for i in self.ids:
#             self.data[i] = {'train_inputs': data['train_inputs'], 'train_targets': data['train_targets'][i], 'label_size': data['label_size']}
#             self.models[i] = lstm_model(data=self.data[i], model_id = i)
#         self.separated_output = [i.layers[-1].outputs for i in self.models.values()]
#         self.inputs = tf.concat(self.separated_output, 1)
# #         print(self.inputs)
# #         self.graph = graph
        
#         with graph.as_default():
#             with tf.name_scope('multi_model'):
#                 self.rebuild_target_placeholder = tf.placeholder(tf.float32, [None, 134])

#                 self.layers = []
#                 self.layers.append(affine_layer(self.inputs, [135, self.hidden_unit1], tf.nn.relu))
#                 self.layers.append(affine_layer(self.layers[-1].outputs, [self.hidden_unit1, 134], tf.nn.relu))

#                 self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.layers[-1].outputs, labels=self.rebuild_target_placeholder))
#                 self.optimizer = tf.train.AdamOptimizer
#                 self.training = self.optimizer(self.learning_rate).minimize(self.loss)
#                 self.accuracy = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(self.layers[-1].outputs, 1), tf.argmax(self.rebuild_target_placeholder, 1)), tf.float32))

#                 self.sess = tf.Session()
#                 merged = tf.summary.merge_all()
#                 writer = tf.summary.FileWriter('./output',self.sess.graph) 
#                 self.sess.run(tf.global_variables_initializer())
            
        
#     def train(self, iteration = 100, interval = 5):
        
#         with graph.as_default():
#             with tf.name_scope('multi_model'):
#             for i in range(iteration):
#                 accs = 0.0
#                 errs = 0.0
#                 ns = 0
#                 for key, j in zip(self.data[0]['train_inputs'], range(len(self.data[0]['train_inputs']))):
#                     train_inputs = np.array(self.data[0]['train_inputs'][key])
#                     train_targets = np.array(self.rebuild_target['train_targets'][key])
#                     batch_size = key
#                     feed_dict = {self.rebuild_target_placeholder: np.array(train_targets, dtype=np.float32)}
#                     _, acc, err = self.sess.run([self.training, self.accuracy, self.loss], feed_dict = feed_dict)
#                     accs += acc
#                     errs += err
#                     ns += train_inputs.shape[0]
#                 accs /= ns
#                 errs /= ns
#                 print('Training Epoch: %d, acc: %f, loss: %f '%(i, accs, errs))
#             if i % interval:
#                 accs = 0.0
#                 errs = 0.0
#                 ns = 0
#                 for key, j in zip(self.data[0]['valid_inputs'], range(len(self.data[0]['valid_inputs']))):
#                     valid_inputs = np.array(self.data[0]['valid_inputs'][key])
#                     valid_targets = np.array(self.rebuild_target['valid_targets'][key])
#                     batch_size = key
#                     feed_dict = {self.rebuild_target_placeholder: np.array(train_targets, dtype=np.float32)}
#                     _, acc, err = self.sess.run([self.training, self.accuracy, self.loss], feed_dict = feed_dict)
#                     accs += acc
#                     errs += err
#                     ns += train_inputs.shape[0]
#                 accs /= ns
#                 errs /= ns
#                 print('------------------------------------------------------')
#                 print('Validation Epoch: %d, acc: %f, loss: %f '%(i, accs, errs))
#                 print('------------------------------------------------------')

In [23]:
# id = 0
# data_all = {'train_inputs': data, 'train_targets': label_all[id], 'label_size': sizes}
# graph = tf.Graph()
# model1 = lstm_model(data=data_all, model_id=id, n_affine=1, graph_name = graph)
# model1.train()